In [4]:
import math


#Read the data into main_data
main_data = {}
with open('data.txt', 'r') as f:
    x = f.read().split()

for i in x:
    if not ("," in i):
        main_data[int(i)] = {}
        current_entry = int(i)
    else:
        connection = i.split(",")
        main_data[current_entry][int(connection[0])] = int(connection[1])

In [78]:
class Heap:
    def __init__(self, container = []):
        self.container = container
    
    def insert(self, length, vertex):        
        self.container.append((length, vertex))
        
        #bubble up
        index = len(self.container) - 1
        while(index > 0):
            parent_index = math.ceil(index/2) - 1
            if(self.container[parent_index][0] > self.container[index][0]):
                self.container[parent_index], self.container[index] = self.container[index], self.container[parent_index]
            index = parent_index
        
    
    def extract_min(self):
        #extract the root element, use the last element replace the root and then bubble down
        result = self.container[0]
        self.container[0] = self.container[-1]
        self.container.pop(-1)
        
        #bubble down
        index = 0
        while(index*2 + 1 < len(self.container)): #The parent still has child
            left_child_index = index*2 + 1
            right_child_index = index*2 + 2
            
            if(right_child_index >= len(self.container)): #The parent only has left child
                if(self.container[index][0] > self.container[left_child_index][0]): #Left child smaller than parent
                    self.container[index], self.container[left_child_index] = \
                    self.container[left_child_index], self.container[index]
                index = left_child_index
            #both side has child
            elif(self.container[left_child_index][0] > self.container[right_child_index][0]):
                if(self.container[index][0] > self.container[right_child_index][0]): #(min)Right child smaller than parent
                    self.container[index], self.container[right_child_index] = \
                    self.container[right_child_index], self.container[index]
                    index = right_child_index
                else:
                    #Correct position, stop (both child larger than parent)
                    index = len(self.container)
            else:
                if(self.container[index][0] > self.container[left_child_index][0]): #(min)Left child smaller than parent
                    self.container[index], self.container[left_child_index] = \
                    self.container[left_child_index], self.container[index]
                    index = left_child_index
                else:
                    #Correct position, stop (both child larger than parent)
                    index = len(self.container)
            
        return result
    
    def update(self, entry, new_value):
        #IMPORTANT!!! since I use heap for dijkstra algorithm, I update since new_value will smaller than the original
        for counter in range(0, len(self.container)):
            if(self.container[counter][1] == entry):
                self.container[counter] = (new_value, entry)
                index = counter
        
        #bubble up from index
        while(index > 0):
            parent_index = math.ceil(index/2) - 1
            if(self.container[parent_index][0] > self.container[index][0]):
                self.container[parent_index], self.container[index] = self.container[index], self.container[parent_index]
            index = parent_index
        
        
    def __str__(self):
        return str(self.container)
    
    def get_copy(self):
        new_heap = Heap(self.container.copy())
        return new_heap
        
    def search(self, entry):
        #This function will return the current length from start vertex to entry vertex
        for edge in self.container:
            if(edge[1] == entry):
                return edge[0]


In [83]:
def get_min(current_map, reached):
    copy_map = current_map.copy()
    for key in reached:
        copy_map.pop(key)
    return min(copy_map, key=copy_map.get)

def dijkstra(edge,start=1):
    #construct map
    current_map = {}
    for i in range(1,201):
        current_map[i] = 1000000
    current_map[start] = 0
    
    reached = set()
    while(reached != set(range(1,201))):
        entry = get_min(current_map, reached)
        reached.add(entry)
        for key,value in edge[entry].items():
            if(current_map[key] > current_map[entry] + value):
                current_map[key] = current_map[entry] + value
    
    return current_map

In [84]:
def heap_min(heap_map, reached):
    heap_map = heap_map.get_copy() #get a copy of the heap
    
    new_edge = heap_map.extract_min()
    while(new_edge[1] in reached):
        new_edge = heap_map.extract_min()
    
    return new_edge

def dijkstra_heap(edge,start=1):
    #construct map
    current_map = Heap()
    for i in range(1,201):
        if(i == start): current_map.insert(0, i)
        else: current_map.insert(1000000, i)
    
    reached = set()
    while(reached != set(range(1,201))):
        entry = heap_min(current_map, reached)[1]
        reached.add(entry)
        for key,value in edge[entry].items():
            if(current_map.search(key) > current_map.search(entry) + value):
                current_map.update(key, current_map.search(entry) + value)
    
    return current_map


In [85]:
#Normal solution (O(mn))
target = [7,37,59,82,99,115,133,165,188,197]
result = dijkstra(main_data, 1)
value = []
for i in target:
    value.append(result[i])
print(value)

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]


In [86]:
#Heap solution (O(mlogn))
result = dijkstra_heap(main_data, 1)
accessed = set()
min_list = {}
for i in range(0,11):
    entry = get_min(result,accessed)
    accessed.add(entry)
    min_list[entry] = result[entry]
print(min_list)

AttributeError: 'Heap' object has no attribute 'copy'